In [ ]:
using Graphs
using GraphPlot
using Combinatorics
using GraphIO
using Plots
using Statistics

function density(g)
    ne(g) / nv(g)
end

In [ ]:
graphs = loadgraphs("datasets/graphs.lg")

Distribution of size of graphs:

In [ ]:
histogram([nv(g) for (key, g) in graphs])

Choosing a k value:

In [ ]:
K = 15

# Preprocessing

In [ ]:
"Estimate the DamkS using the greedy approach"
function greedy_damks_estimate(g, k)
    h = deepcopy(g)
    # Remove vertices with minimum degree until the graph has `k` vertices left
    for c in 1:(nv(g) - k)
        min_d, min_v = findmin(v -> length(neighbors(h, v)), vertices(h))
        rem_vertex!(h, min_v)
    end
    # Remove vertices to check for better density
    max_d = density(h)
    best_g = deepcopy(h)
    for c in 1:min(k-1, nv(h))
        min_d, min_v = findmin(v -> length(neighbors(h, v)), vertices(h))
        rem_vertex!(h, min_v)
        if density(h) > max_d
            max_d = density(h)
            best_g = deepcopy(h)
        end
    end
    best_g
end

"Prune vertices that have degree lower than `d`"
function prune_graph(g, d)
    can_rem = true
    h = deepcopy(g)
    while can_rem
        to_be_rem = Vector{Int64}()
        for v in 1:nv(h)
            if length(neighbors(h, v)) < d
                push!(to_be_rem, v)
            end
        end
        can_rem = length(to_be_rem) > 0
        rem_vertices!(h, to_be_rem)
    end
    h
end

Pruning

In [ ]:
pruned = Dict(key => prune_graph(g, density(greedy_damks_estimate(g, K))) for (key, g) in graphs)

Size distribution after pruning

In [ ]:
histogram([nv(g) for (key, g) in pruned])

In [ ]:
pruned_2 = filter(((gn, g),) -> nv(g) <= 30, pruned)

# Densest At-most-k Subgraph

### 1. True DamkS

In [ ]:
function exact_daks(g, k)
    # Exact algorithm to find the densest at-k subgraph
    best_g = nothing
    max_d = 0
    for vec in combinations(1:nv(g), k)
        h, vmap = induced_subgraph(g, vec)
        if density(h) > max_d
            best_g = h
            max_d = density(h)
        end
    end
    best_g
end

function exact_damks(g, k)
    # Exact algorithm to find the densest at-most-k subgraph
    best_g = nothing
    max_d = 0
    for ks in 2:k
        h = exact_daks(g, ks)
        if density(h) > max_d
            max_d = density(h)
            best_g = h
        end
    end
    best_g
end

function true_densities(graphs, k)
    # Exact algorithm to find the density of DamkS
    res = Dict{String, Float64}()
    for (name, g) in graphs
        res[name] = exact_damks(g, k)
    end
    res
end

Test on graph "1"

In [ ]:
# g = graphs["1"]
g = pruned["1"]
# gplot(g)
h = exact_damks(g, K)
print("Density = ")
println(density(h))
gplot(h)

Run

In [ ]:
target = true_densities(pruned_2, K)